In [ ]:
#%run initplot.py
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
from scipy.integrate import odeint, ode
import matplotlib.pyplot as plt
from scipy import stats

from math import factorial
def bn(N,k):
    return float(factorial(N))/(factorial(k)*factorial(N-k))

from titlecase import titlecase
nicenames = {'evangelical':'evangelical','coolkids':'cool kids','snobs':'snobs','conformity2':'conformists','conformity3':'conformists'}
from __future__ import unicode_literals

legendkeys = {u'i-i->n':r'$i\rightarrow i \;\;to\;\; i\rightarrow s$',
              u'i-n->i':r'$i\rightarrow s \;\;to\;\; i\rightarrow i$',
              u'n-i->n':r'$s\rightarrow i \;\;to\;\; s\rightarrow s$',
              u'n-n->i':r'$s\rightarrow s \;\;to\;\; s\rightarrow i$',
             }
rewire_types = ['i-i->n','n-i->n','i-n->i','n-n->i',]
allmodels = ['evangelical','coolkids','snobs']


In [ ]:
def derivs(prms, t, beta, case):
    I, SI, II, _,_,_,_ = prms ; S  = 1.0 - I ; SS = 1.0 - SI - II
    
    #if np.isclose(I, 0.0) or np.isclose(I, 1.0) or np.isclose(II, 1.0): return 0., 0., 0., 0., 0., 0., 0.
    

    # # Slightly simpler rewiring
    #rewire_si2ii = I*(1-np.exp(-E*SI/(N*I)))/E
    #rewire_ss2si = S*(1-np.exp(-E*2*SS/(N*S)))/E
    
    alpha_I_II = 2*II/(SI + 2*II)   if II > 0 else 0
    alpha_I_SI = 1. - alpha_I_II
    alpha_S_SS = 2*SS/(SI + 2*SS) if SS > 0 else 0
    alpha_S_SI = 1. - alpha_S_SS
    degI = E*(SI + 2*II)/(N*I)    if I > 0 else 0
    degS = E*(SI + 2*SS)/(N*S)    if S > 0 else 0
    
    r_I_fromItoS = 0. #np.nan 
    r_S_fromItoS = 0. #np.nan
    r_I_fromStoI = 0. #np.nan
    r_S_fromStoI = 0. #np.nan
    
    if case == 'evangelical':
        rewire_si2ii = -I*(1-(1-alpha_I_II)**degI)/E
        rewire_ss2si = S*(1-(1-alpha_S_SS)**degS)/E
        r_I_fromItoS = -rewire_si2ii
        r_S_fromStoI = rewire_ss2si
        
    elif case == 'coolkids':
        rewire_si2ii = I*(1-(1-alpha_I_SI)**degI)/E
        rewire_ss2si = S*(1-(1-alpha_S_SS)**degS)/E
        r_I_fromStoI = rewire_si2ii
        r_S_fromStoI = rewire_ss2si
        
    elif case == 'snobs':
        #print SI, SS, S
        cutoff = int(np.ceil((degS+2)/4))
        degSint = int(degS)
        sAssP = stats.binom.cdf(cutoff, degSint, alpha_S_SI) - stats.binom.cdf(0, degSint, alpha_S_SI)
        sDissP = stats.binom.cdf(degS - cutoff, degSint, alpha_S_SS) - stats.binom.cdf(0, degSint, alpha_S_SS)
                
        """
        if 0.0 < alpha_S_SI < 1.0:
            sAssP = stats.binom.cdf(cutoff, degSint, alpha_S_SI) - stats.binom.cdf(0, degSint, alpha_S_SI)
        else:
            sAssP = 0.
        if 0.0 < alpha_S_SS < 1.0:
            sDissP = stats.binom.cdf(degS - cutoff, degSint, alpha_S_SS) - stats.binom.cdf(0, degSint, alpha_S_SS)
        else:
            sDissP = 0.
        """
        #print '%0.3f %0.3f'%(sDissP, sAssP)
        #asdf
        rewire_ss2si = (S*sDissP - S*sAssP)/E
        rewire_si2ii = I*(1-(1-alpha_I_SI)**degI)/E
        r_I_fromStoI = rewire_si2ii
        r_S_fromItoS = sAssP * S/E
        r_S_fromStoI = sDissP * S/E
        #rewire_ss2si = S*(1-(1-alpha_S_SS)**degS)/E
        """
        print S*(1-(1-alpha_S_SS)**degS)/E, S*sDissP/E , S*sAssP/E
        print np.ceil((degS+2)/4), degS, alpha_S_SI, 'alpha_S_SS=', alpha_S_SS
        print 'I=', I, 'SI=', SI, 'II=', II
        print 'SS=',SS # , *SS/(SI + 2*SS)'
        
        print 'degS=',degS
        print sAssP
        maxK = int(np.ceil((degS+2)/4))
        print np.sum([bn(int(degS),i) * (alpha_S_SI**i) * (1 - alpha_S_SI)**(degS-i) for i in range(1, maxK)])
        
        print '----'
        print maxK
        print stats.binom.cdf(maxK, int(degS), alpha_S_SI) 
        print np.sum([bn(int(degS),i) * (alpha_S_SI**i) * (1 - alpha_S_SI)**(degS-i) for i in range(0, maxK)])
        print '/////'
        print stats.binom.cdf(0, degS, alpha_S_SI)
        print bn(int(degS),0) * (alpha_S_SI**0) * (1 - alpha_S_SI)**(degS-0)
        raise Exception()
        """
        
    else:
        raise Exception('unknown case')
    
    dI  = beta * SI
    dSI = beta * SI * (2*SS - SI)/S - rewire_si2ii + rewire_ss2si
    dII = beta * SI * (SI/S) + rewire_si2ii

    return np.array([dI, dSI, dII, r_I_fromItoS, r_S_fromItoS, r_I_fromStoI, r_S_fromStoI])

"""
def myodeint(func, y0, t, args=None):
    func2 = lambda t, y, args: func(y, t, *args)   # odeint has these the other way :/
    r = ode(func2).set_integrator('vode').set_initial_value(y0, t=t[0]).set_f_params(args)
    dt = 1
    while r.successful() and r.t < t1:
        r.integrate(r.t+dt)

    y = [sol.integrate(tp) for tp in t[1:]]
    y.insert(0, y0)
    return np.array(y)
"""

def integrate(y0, timepoints, beta, cmodel):
    # y0=[initI, (1.-initI)*initI, initI**2, 0, 0, 0, 0]
    if False:
        #traj = myodeint(derivs, y0, timepoints, args=(beta,cmodel))
        traj = odeint(derivs, y0, timepoints, args=(beta,cmodel)) # , rtol=1e-10, atol=1e-10, )
    else:
        state=np.array(y0)
        traj=[]
        traj.append(state.copy())
        for tndx, t in enumerate(timepoints[1:]):
            state += (timepoints[tndx+1]-timepoints[tndx])*np.array(derivs(state, t, beta, cmodel))
            #print state[0:3]
            if state[0]>1: state[0] = 1.
            if state[1]>1: state[1] = 1.
            if state[2]>1: state[2] = 1.
            if state[0]<0: state[0] = 0.
            if state[1]<0: state[1] = 0.
            if state[2]<0: state[2] = 0.
            #state[0:3] = np.max(np.zeros(3), state[0:3])
            #state[0:3] = np.min(np.ones(3), state[0:3])
            traj.append(state.copy())
        traj=np.array(traj)
    return traj

traj = integrate(y0, np.linspace(0, 100, 500), 1e-4, 'snobs')
#print np.diff(traj,axis=0)[0,:]
#asdadsf



In [ ]:
E = 8000.     # number of edges
N = 1000.     # number of people
initI = .05 # 1e-2  # initial fraction infected
timepoints = np.linspace(0, 10000, 10000)   # time steps

#finalT = 20000
#timepoints = np.exp(np.linspace(0, np.log(finalT+1), 2000))-1
beta  = .0002 # 5e-4
y0=np.array([initI, (1.-initI)*initI, initI**2, 0, 0, 0, 0])

In [ ]:
rdicts = {}
#beta = 1
for cmodel in allmodels:
    traj = integrate(y0, timepoints, beta, cmodel)
    if False:
        plt.figure(figsize=(10,4))
        plt.subplot(1,2,1)
        plt.plot(timepoints, traj)
        plt.legend(['I', 'SI', 'II'])
        plt.xlabel('Time')
        plt.ylim([0,1])
        plt.subplot(1,2,2)
        plt.plot(timepoints[:-1], np.diff(traj,axis=0))
        plt.legend(['dI', 'dSI', 'dII'])
        plt.xlabel('Time')

    if True:
        plt.figure()
        plt.plot(timepoints, traj[:,0])
        #print traj.shape
        #print  traj[:,0]
        plt.ylim([0,1])

        rdicts[cmodel]= { 'num_infected' : traj[:,0] ,
                          'rewire_counts_list': np.diff(traj,axis=0)[:,-4:],
                          'rewire_types': ['i-i->n','n-i->n','i-n->i','n-n->i',]
                        }
        print cmodel,  np.diff(traj,axis=0)[:,-4:][0,:]

In [ ]:
plt.rcParams['font.size']         = 34*.4

cnet = 'ER'
#mx_init, is_infected_init = inits[cnet]['mx_init'], inits[cnet]['is_infected_init']
#is_infected_init
#rdicts = rdicts_db[cnet]
ncols = 5

#plotitems = rdicts.keys()
plotitems = allmodels
ndicts = len(plotitems)
f, axmx = plt.subplots(4, 3, figsize=(10,4*ndicts)) # , sharex='col', sharey='row')
f.subplots_adjust(hspace=.35, wspace=.35)

axmx =axmx.T
for ndx, axrow in enumerate(axmx):
    cmodel = plotitems[ndx]
    num_inf_ax, rewiring_ax, degree_ax, blk_ax = axrow
    if cmodel not in rdicts:
        continue
    rdict = rdicts[cmodel]
    colndx = 1
    print ndx, cmodel

    plt.sca(num_inf_ax)
    plt.plot(timepoints, np.array(rdict['num_infected']),color='k')
    plt.title(titlecase(nicenames[cmodel]))
    if ndx == 0: 
        plt.ylabel('Proportion infected')
    plt.xlabel('Iteration')
    from matplotlib.ticker import MaxNLocator
    plt.gca().xaxis.set_major_locator( MaxNLocator(nbins = 3) )
    #plt.locator_params(nbins=4,axis='x')

    plt.ylim([0, 1])
    plt.xlim([np.min(timepoints), np.max(timepoints)])
    # plt.plot(rdict['assortativity']);plt.title('%s - inf assortativity'%t)
    plt.sca(rewiring_ax)
    rws = np.array(rdict['rewire_counts_list'])
    #rwsndx, rwtypes = zip(*[(i, rt) for i, rt in enumerate(rewire_types) if rt[-4:]!='none'])
    #if ndx == 0: 
    #    plt.ylabel('Rewirings')
    ##plotrws = rws[:,list(rwsndx)]
    #plotrws = rws

    plt.plot(timepoints[1:], rws)
    
    plt.xlabel('Iteration')
    plt.gca().xaxis.set_major_locator( MaxNLocator(nbins = 3) )

    #plt.ylim([0, plt.ylim()[1]*1.1])
    #if ndx == 0:
    #    labels = [str(int(item)) if ndx <1 else '' for ndx, item in enumerate(plt.gca().get_xticks())]
    #    plt.gca().set_xticklabels(labels)

    #    legend = plt.legend([legendkeys.get(rwt, rwt) for rwt in rwtypes], bbox_to_anchor=(1.1,.28), fontsize=12)
    #    legend.get_frame().set_facecolor('white')
    continue 

    plt.sca(degree_ax)
    if False:
        densemx = np.array(rdict['mx'].todense()) if ss.issparse(rdict['mx']) else rdict['mx']
        degs=densemx.sum(axis=0)
        plt.hist(degs, bins=20)  # TODO : large degrees, with frequency 1, not showing up at all
        plt.locator_params(nbins=4,axis='x')
        plt.xlabel('Degree')
        if ndx == 0:
            plt.ylabel('Frequency')

        plt.sca(blk_ax)
        if N < 2000:
            if True:
                is_infected_ixs = np.flatnonzero(rdict['is_infected'])
                is_not_infected_ixs = np.flatnonzero(~rdict['is_infected'])

                infdegs   =np.argsort(densemx[rdict['is_infected'],:].sum(axis=1))[::-1]
                notinfdegs=np.argsort(densemx[~rdict['is_infected'],:].sum(axis=1))[::-1]
                #print infdegs.max(), len(is_infected_ixs)
                #print notinfdegs.max(), len(is_not_infected_ixs),is_not_infected_ixs
                ixs = list(is_infected_ixs[infdegs]) + list(is_not_infected_ixs[notinfdegs])

                im = densemx[ixs,:][:,ixs]
                #print densemx.shape, im.shape, ixs
                #adsf
                #plt.imshow(im, cmap='Greys', aspect='auto', interpolation='nearest')
                im3 = np.zeros((N,N,3))
                im3 += (1-im[:,:,None])
                r=np.zeros((N,N))[:,:,None]+np.array([0,.1,.1])[None,None,:]
                r[len(is_infected_ixs):,:]=0
                r[:,len(is_infected_ixs):]=0
                im3 -= r
                im3[im3<0]=0
                if len(is_infected_ixs) < N:
                    r=np.zeros((N,N))[:,:,None]+np.array([.1,.1,0])[None,None,:]
                    r[:len(is_infected_ixs),:]=0
                    r[:,:len(is_infected_ixs)]=0
                    im3 -= r
                    im3[im3<0]=0

                plt.imshow(im3,  interpolation='nearest')


                plt.gca().grid(b=False)
                if False:
                    if len(is_infected_ixs) != N:
                        plt.plot([0, N], [len(is_infected_ixs), len(is_infected_ixs)], color='k', linestyle='-', linewidth=1)
                        plt.plot( [len(is_infected_ixs), len(is_infected_ixs)], [0, N], color='k', linestyle='-', linewidth=1)
                plt.xlim([0,N])
                plt.ylim([N,0])
                plt.xticks([0,N], ['1',str(N)])
                plt.yticks([0,N], ['1',str(N)])
                plt.xlabel('Adjacency Matrix Node')
                if ndx == 0:
                    plt.ylabel('Node')

    """
    plt.sca(blk_ax)
    #plt.hist(rdict['mx'].sum(axis=0), bins=20)
    #ixs = list(np.flatnonzero(is_infected_init)) + list(np.flatnonzero(~is_infected_init))
    #plt.imshow(rdict['mx'][ixs,:][:,ixs], cmap='Greys', aspect='auto', interpolation='nearest')
    plt.scatter(rdict['time_infected'], rdict['mx'].sum(axis=0))
    plt.xlabel('time inf')
    plt.ylabel('degree')
    """

    print
fname='out/gridfig1-%s.pdf'%cnet
if False:
    print 'saving', fname
    plt.savefig(fname, facecolor='white')
    

In [ ]:
finalrates = {}
betavals = 10.**np.linspace(-6, 0, 20)
print betavals


timepoints2 = np.linspace(0, 20000, 10000)   # time steps
for cmodel in allmodels:
    r = []
    for beta in betavals:
        print cmodel, beta
        traj = integrate(y0, timepoints2, beta, cmodel)
        f = np.nansum(np.diff(traj, axis=0), axis=0)
        r.append(f)
    finalrates[cmodel] = np.array(r)
         

In [ ]:
f, axmx = plt.subplots(2, 3, figsize=(12,8)) # , sharex='col', sharey='row')
f.subplots_adjust(hspace=.35, wspace=.35)
plt.figure(figsize=(9*4,12*4))
for tndx, cmodel in enumerate(allmodels):
    plt.sca(axmx[0,tndx])
    plt.semilogx(betavals, finalrates[ cmodel ][:,0], color='k')
    plt.sca(axmx[1,tndx])
    plt.semilogx(betavals, finalrates[ cmodel ][:,-4:])

In [ ]:
    
    #plt.figure()
    #print finalrates[cmodel]
    plt.sca(axmx[0,tndx])
    plt.semilogx(betavals, finalrates[ cmodel ][:,0], color='k')
    #continue
    plt.sca(axmx[1,tndx])
    #all_rwts = [ 'i-i->n', 'n-i->n', 'i-n->i',  'n-n->i']
    plt.semilogx(betavals, finalrates[ cmodel ][:,-4:])
    continue
    for rwtndx, rwt in enumerate(rewire_types):
        #clist = []
        #
        #for i in range(len(d)):
        #    ix = [ndx for ndx, c in enumerate(d[i][1]['rewire_types']) if c == rwt][0]
        #    clist.append((d[i][0], np.array(d[i][1]['rewire_counts_list'])[-1,ix]))
        #ps, data = zip(*clist) 
        #print finalrates[ cmodel ][:,4+rwtndx]
        plt.semilogx(betavals, finalrates[ cmodel ][:,-4:])
    
    continue
    ps, numinfs = zip(*[(betavals, finalrates[ cmodel ]) for i in range(len(d))])
    #plt.subplot(3,2,tndx*2+1)
    plt.sca(axmx[0,tndx])
    plt.semilogx(ps, np.array(numinfs)/float(N), color='k')
    plt.ylim([0, 1.05*plt.ylim()[1]])
    plt.xlabel(r'$\beta$', fontsize=18)
    plt.ylabel('Proportion infected')
    plt.vlines(0.0002, plt.ylim()[0], plt.ylim()[1], colors='k', linestyles='--', linewidth=1)
    plt.title(titlecase(nicenames[cmodel]))
    #plt.subplot(3,2,tndx*2+2)
    plt.sca(axmx[1,tndx])
    all_rwts = [ 'i-i->n', 'n-i->n', 'i-n->i',  'n-n->i']
    for rwt in all_rwts:
        clist = []

        for i in range(len(d)):
            ix = [ndx for ndx, c in enumerate(d[i][1]['rewire_types']) if c == rwt][0]
            clist.append((d[i][0], np.array(d[i][1]['rewire_counts_list'])[-1,ix]))
        ps, data = zip(*clist) 
        plt.semilogx(ps, data)
    plt.xlabel(r'$\beta$', fontsize=18)
    plt.ylabel('Total number rewirings')
    plt.vlines(0.0002, plt.ylim()[0], plt.ylim()[1], colors='k', linestyles='--', linewidth=1)
    if tndx == 0:
        legend = plt.legend([legendkeys.get(rwt, rwt) for rwt in all_rwts], bbox_to_anchor=(1.06,1), fontsize=12)
        legend.get_frame().set_facecolor('white')

#plt.savefig('out/ptransmit_sweep_v2.pdf', bbox_inches='tight',facecolor='white')

In [ ]:
plt.semilogx(betavals, finalrates[ cmodel ][:,4])
    